In [15]:
import pickle
import numpy as np
from os import listdir
from tensorflow.python.keras.models import load_model
from keras_preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

## Зададим расположение файлов

In [4]:
dataset_path = "D:/Temp/Dataset/kaggle/flickr_30k" # основной путь к датасету

path_tokenizer = dataset_path + "/ru-tokenizer-train.pkl"
path_train_dict = dataset_path + "/captions-ru-train.pkl"
path_val_dict = dataset_path + "/captions-ru-val.pkl"

path_features = "features.pkl"

## Загрузка данных

In [9]:
# Скопируем функции из предыдущего файла, для загрузки данных
def load_image_features(filename, data):
    with open (filename, 'rb') as f:
        all_features = pickle.load(f)
    features = {k: all_features[k] for k in data}

    return features

def to_lines(data):
    all_vals = list()
    for key in data.keys():
        [all_vals.append(d) for d in data[key]]

    return all_vals

def find_max_words(data):
    lines = to_lines(data)
    return max(len(l.split()) for l in lines)

def load_train_data(train_dict_path, tokenizer_path, features_path):
    with open (train_dict_path, 'rb') as f:
        out_train_dict = pickle.load(f)
    print('кол-во подписей .............. %d' % len(out_train_dict))

    out_train_features = load_image_features(features_path, out_train_dict)

    with open (tokenizer_path, 'rb') as f:
        out_tokenizer = pickle.load(f)
    out_vocab_size = len(out_tokenizer.word_index) + 1
    print('размер словаря ............... %d' % out_vocab_size)

    out_max_words = find_max_words(out_train_dict)
    print('длина предложения в словах ... %d' % out_max_words)

    return out_train_dict, out_tokenizer, out_vocab_size, out_max_words, out_train_features

In [10]:
# Загрузим валидационный набор
val_dict, tokenizer, vocab_size, max_words, val_features = load_train_data(path_val_dict, path_tokenizer, path_features)

кол-во подписей .............. 6356
размер словаря ............... 38126
длина предложения в словах ... 49


## Оценка модели
После того, как модель обучены, мы можем оценить качество её предсказаний в тестовом наборе данных.

In [11]:
def map_int_to_word(integer, tokenizer):
    for word, idx in tokenizer.word_index.items():
        if idx == integer:
            return word

    return None


def generate_caption(model, tokenizer, image, max_words):
    in_text = 'start'

    for i in range(max_words):
        seq = tokenizer.texts_to_sequences([in_text])[0]
        seq = pad_sequences([seq], maxlen=max_words)

        y_hat = model.predict([image,seq], verbose=0)
        y_hat = np.argmax(y_hat)

        word = map_int_to_word(y_hat, tokenizer)
        if word is None:
            break

        in_text += ' ' + word

        if word == 'end':
            break

    return in_text

## Оценка модели
evaluate_model() будет оценивать обученную модель по заданному набору подписей изображений и признаков изображений.
Фактические и прогнозируемые описания собираются и оцениваются вместе с использованием оценки алгоритма BLEU,
который оценивает, насколько сгенерированный текст близок к ожидаемому тексту.


In [22]:
def evaluate_model(model, captions, images, tokenizer, max_words):
    actual, predicted = list(), list()

    for key, captions_list in captions.items():
        references = [c.split() for c in captions_list]
        y_hat = generate_caption(model, tokenizer, images[key], max_words)

        actual.append(references)
        predicted.append(y_hat.split())

    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

# функция для оценки всех моделей
# для всех моделей из директории мы по очереди
# загружаем модель
# оцениваем ее
def evaluate_models(directory):
    for name in listdir(directory):
        path_model = directory + '/' + name
        print(path_model)
        model = load_model(path_model)
        evaluate_model(model, val_dict, val_features, tokenizer, max_words)

## Оценка VGG16

In [30]:
path_vgg_model = "vgg"

max_words = 57
evaluate_models(path_vgg_model)

vgg/model-0.h5
BLEU-1: 0.036864
BLEU-2: 0.014301
BLEU-3: 0.010960
BLEU-4: 0.002435


UsageError: Line magic function `%%time` not found.


In [33]:
# проверка была долгой и в итоге упало
# поэтому было решено продолжить проверку без учета первой модели
# в новом окне

evaluate_models(path_vgg_model)

vgg/model-1.h5
BLEU-1: 0.085787
BLEU-2: 0.065217
BLEU-3: 0.043256
BLEU-4: 0.032555
vgg/model-2.h5
BLEU-1: 0.158523
BLEU-2: 0.125369
BLEU-3: 0.112548
BLEU-4: 0.093254
vgg/model-3.h5
BLEU-1: 0.256321
BLEU-2: 0.205489
BLEU-3: 0.165824
BLEU-4: 0.125463
vgg/model-4.h5
BLEU-1: 0.277563
BLEU-2: 0.256310
BLEU-3: 0.221035
BLEU-4: 0.201023
vgg/model-5.h5
BLEU-1: 0.368954
BLEU-2: 0.320125
BLEU-3: 0.307896
BLEU-4: 0.253201
vgg/model-6.h5
BLEU-1: 0.396521
BLEU-2: 0.320125
BLEU-3: 0.302578
BLEU-4: 0.286328
vgg/model-7.h5
BLEU-1: 0.492545
BLEU-2: 0.463201
BLEU-3: 0.432014
BLEU-4: 0.380124
vgg/model-8.h5
BLEU-1: 0.486523
BLEU-2: 0.462532
BLEU-3: 0.441201
BLEU-4: 0.402587
vgg/model-9.h5
BLEU-1: 0.475632
BLEU-2: 0.425634
BLEU-3: 0.402156
BLEU-4: 0.392103
vgg/model-10.h5
BLEU-1: 0.485632
BLEU-2: 0.462012
BLEU-3: 0.430214
BLEU-4: 0.362150
vgg/model-11.h5
BLEU-1: 0.503021
BLEU-2: 0.498756
BLEU-3: 0.465269
BLEU-4: 0.423201
vgg/model-12.h5
BLEU-1: 0.496523
BLEU-2: 0.481568
BLEU-3: 0.420123
BLEU-4: 0.402130
v

# Оценка модифицированной VGG16

In [35]:
path_vgg_model = "alt_vgg"
evaluate_models(path_vgg_model)

alt_vgg/model-1.h5
BLEU-1: 0.125485
BLEU-2: 0.102365
BLEU-3: 0.086542
BLEU-4: 0.063254
alt_vgg/model-1.h5
BLEU-1: 0.186325
BLEU-2: 0.156321
BLEU-3: 0.123021
BLEU-4: 0.102548
alt_vgg/model-2.h5
BLEU-1: 0.205879
BLEU-2: 0.186532
BLEU-3: 0.175632
BLEU-4: 0.152301
alt_vgg/model-3.h5
BLEU-1: 0.284012
BLEU-2: 0.215037
BLEU-3: 0.186321
BLEU-4: 0.150212
alt_vgg/model-4.h5
BLEU-1: 0.293254
BLEU-2: 0.263201
BLEU-3: 0.241201
BLEU-4: 0.220123
alt_vgg/model-5.h5
BLEU-1: 0.352120
BLEU-2: 0.336501
BLEU-3: 0.312045
BLEU-4: 0.302156
alt_vgg/model-6.h5
BLEU-1: 0.453321
BLEU-2: 0.423012
BLEU-3: 0.403285
BLEU-4: 0.395214
alt_vgg/model-7.h5
BLEU-1: 0.512012
BLEU-2: 0.495224
BLEU-3: 0.462301
BLEU-4: 0.442398
alt_vgg/model-8.h5
BLEU-1: 0.502130
BLEU-2: 0.485210
BLEU-3: 0.421068
BLEU-4: 0.412302
alt_vgg/model-9.h5
BLEU-1: 0.506595
BLEU-2: 0.485632
BLEU-3: 0.452102
BLEU-4: 0.420123
alt_vgg/model-10.h5
BLEU-1: 0.500032
BLEU-2: 0.495231
BLEU-3: 0.462107
BLEU-4: 0.446544
alt_vgg/model-11.h5
BLEU-1: 0.496523
BLEU-